# 2021 Baseball Stats

Instructions:

- Download the batter and pitcher stat CSV files (split and complete)

- Upload the downloaded CSV files to the session storage



# Imports and startup

In [1]:
# Necessary imports

import math
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

from datetime import datetime
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
from ipywidgets import interactive
import ipywidgets as widgets
from IPython.display import display, clear_output
from ipywidgets import interact, Layout
from bokeh.io import output_notebook
from prettytable import PrettyTable
import locale
import warnings

warnings.filterwarnings('ignore')

%matplotlib inline

# For number formatting 
locale.setlocale(locale.LC_ALL, '') 

# Call once to configure Bokeh to display plots inline in the notebook
output_notebook()

# Style of plots
matplotlib.style.use('seaborn')

#quick startup
btc = pd.read_csv('MLB-stats-2021-b.csv') #complete season for batters
bts = pd.read_csv('MLB-stats-2021-b-split.csv') #split season (played for different teams) for batters
ptc = pd.read_csv('MLB-stats-2021-p.csv') #complete season for pitchers
pts = pd.read_csv('MLB-stats-2021-p-split.csv') #split season for pitchers 

#store player index values
bi = pd.Series(btc['Name'], index = range(0,1508)) #batter index
pi = pd.Series(ptc['Name'], index = range(0,909)) #pitcher index

#split pitchers into starters and relievers 
#(qualified as a starter if they started in over half of their games)
startersIndex = []
relieversIndex = []
index = 0
for player in pi:
  pitcher =  ptc.loc[index]
  gs = pitcher.at['GS']
  g = pitcher.at['G']
  if(g/gs < 2):
    startersIndex.append(index)
  else:
    relieversIndex.append(index)
  index = index+1
starters = ptc.loc[startersIndex]
relievers = ptc.loc[relieversIndex]

Loading BokehJS ...

# Batter Functions

Print out all batters (complete stats)

In [6]:
display(btc)

,Name,Team,G,AB,PA,H,1B,2B,3B,HR,...,HBP,SF,SH,GDP,SB,CS,AVG,OBP,SLG,OPS
0,Tommy Hunter,NYM,4,1,1,1,1,0,0,0,...,0,0,0,0,0,0,1.0,1.0,1.0,2.0
1,Daniel Bard,COL,67,1,1,1,1,0,0,0,...,0,0,0,0,0,0,1.0,1.0,1.0,2.0
2,Connor Overton,- - -,9,1,1,1,1,0,0,0,...,0,0,0,0,0,0,1.0,1.0,1.0,2.0
3,Jose Quintana,- - -,29,1,2,1,1,0,0,0,...,0,0,0,0,0,0,1.0,1.0,1.0,2.0
4,Josh Lowe,TBR,2,1,2,1,1,0,0,0,...,0,0,0,0,1,0,1.0,1.0,1.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1503,Garrett Crochet,CHW,54,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.0,0.0,0.0,0.0
1504,Spencer Strider,ATL,2,1,2,0,0,0,0,0,...,0,0,1,0,0,0,0.0,0.0,0.0,0.0
1505,Kohei Arihara,TEX,10,2,2,0,0,0,0,0,...,0,0,0,0,0,0,0.0,0.0,0.0,0.0
1506,Hyeon-jong Yang,TEX,12,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.0,0.0,0.0,0.0


Display a team's stats individually

In [7]:
nyyB = bts.loc[bts['Team'] == 'NYY']
nyyP = pts.loc[pts['Team'] == 'NYY']
nyyB

,Name,Team,G,AB,PA,H,1B,2B,3B,HR,...,SF,SH,GDP,SB,CS,AVG,OBP,SLG,OPS,playerid
34,Estevan Florial,NYY,11,20,25,6,3,2,0,1,...,0,0,0,1,0,0.300,0.440,0.550,0.990,19151
64,Aaron Judge,NYY,148,550,633,158,95,24,0,39,...,5,0,16,6,1,0.287,0.373,0.544,0.916,15640
89,Greg Allen,NYY,15,37,48,10,5,4,1,0,...,1,0,0,5,0,0.270,0.417,0.432,0.849,16623
101,Giancarlo Stanton,NYY,139,510,579,139,85,19,0,35,...,3,0,22,0,0,0.273,0.354,0.516,0.870,4949
164,Jameson Taillon,NYY,29,1,2,0,0,0,0,0,...,0,0,0,0,0,0.000,0.500,0.000,0.500,11674
167,Clarke Schmidt,NYY,2,1,2,0,0,0,0,0,...,0,0,0,0,0,0.000,0.500,0.000,0.500,19899
215,Anthony Rizzo,NYY,49,173,200,43,28,7,0,8,...,2,0,5,2,0,0.249,0.340,0.428,0.768,3473
230,Luke Voit,NYY,68,213,241,51,32,7,1,11,...,0,0,12,0,0,0.239,0.328,0.437,0.764,14811
252,Ryan LaMarre,NYY,9,21,24,4,2,0,0,2,...,0,0,1,1,0,0.190,0.292,0.476,0.768,10700
319,DJ LeMahieu,NYY,150,597,679,160,125,24,1,10,...,5,0,16,4,2,0.268,0.349,0.362,0.711,9874



Sort players by specific stat

In [8]:
bt_sort_H = btc.sort_values(by='H', ascending=False)
bt_sort_H

,Name,Team,G,AB,PA,H,1B,2B,3B,HR,...,HBP,SF,SH,GDP,SB,CS,AVG,OBP,SLG,OPS
61,Trea Turner,- - -,148,595,646,195,130,34,3,28,...,6,4,0,18,32,5,0.328,0.375,0.536,0.911
118,Bo Bichette,TOR,159,640,690,191,131,30,1,29,...,6,4,0,10,25,1,0.298,0.343,0.484,0.828
33,Vladimir Guerrero Jr.,TOR,161,604,698,188,110,29,1,48,...,6,2,0,20,4,1,0.311,0.401,0.601,1.002
338,Whit Merrifield,KCR,162,664,720,184,129,42,3,10,...,4,12,0,12,40,4,0.277,0.317,0.395,0.711
71,Freddie Freeman,ATL,159,600,695,180,122,25,2,31,...,7,2,0,11,8,3,0.300,0.393,0.503,0.896
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1025,Gerrit Cole,NYY,30,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.000,0.000,0.000,0.000
1026,Derek Law,MIN,9,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.000,0.000,0.000,0.000
1027,Eduardo Rodriguez,BOS,32,6,6,0,0,0,0,0,...,0,0,0,1,0,0,0.000,0.000,0.000,0.000
1028,Kyle Crick,PIT,27,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.000,0.000,0.000,0.000


Dropdown menu of stats to display with option for descending or ascending

In [9]:
def sortByStat(stat, direction):
  if direction == 'Ascending':
    directionB = False
  else:
    directionB = True
  return btc.sort_values(by= stat, ascending= directionB)

interact(sortByStat, stat = list(btc.columns), direction = ['Ascending', 'Descending'])

interactive(children=(Dropdown(description='stat', options=('Name', 'Team', 'G', 'AB', 'PA', 'H', '1B', '2B', …

<function __main__.sortByStat(stat, direction)>

Filters out players by a minimum number of at bats

In [10]:
bt_qual = btc.loc[btc['AB'] > 100]

def sortByStat_100ab(stat, direction):
  if direction == 'Ascending':
    directionB = False
  else:
    directionB = True
  if stat == 'Name' or stat == 'Team' or stat == 'G' or stat == 'AB':
    return btc.sort_values(by= stat, ascending= directionB)
  else:
    return bt_qual.sort_values(by= stat, ascending= directionB)

interact(sortByStat_100ab, stat = list(btc.columns), direction = ['Ascending', 'Descending'])

interactive(children=(Dropdown(description='stat', options=('Name', 'Team', 'G', 'AB', 'PA', 'H', '1B', '2B', …

<function __main__.sortByStat_100ab(stat, direction)>

Bar chart for top batters for a certain stat

In [11]:
def bar_plotB(stat, direction):
  bt_sorted = btc
  bt_qual = btc.loc[btc['AB'] > 100]
  if direction == 'Ascending':
    directionB = False
  else:
    directionB = True
  bt_sorted = bt_qual.sort_values(by= stat, ascending= directionB)
  fig = px.bar(bt_sorted.head(100), x='Name', y=stat, width=1300, height=600)
  fig.update_layout(title_text="Top Players in Hitting Stats in MLB (2021)")
  fig.update_xaxes(rangeslider_visible=True)
  fig.show()

interact(bar_plotB, stat=list(btc.columns)[2:], direction = ['Ascending', 'Descending'])

interactive(children=(Dropdown(description='stat', options=('G', 'AB', 'PA', 'H', '1B', '2B', '3B', 'HR', 'R',…

<function __main__.bar_plotB(stat, direction)>

Search for player by name

In [12]:
#exception handling for nonexistent players
try:
  search = input("Enter a player's full name to search for: ")
  searchResult = btc.loc[btc['Name'] == search]
  list(bi).index(search)
  display(searchResult)
except ValueError:
  print('This player does not exist.')

Enter a player's full name to search for: Aaron Judge


,Name,Team,G,AB,PA,H,1B,2B,3B,HR,...,HBP,SF,SH,GDP,SB,CS,AVG,OBP,SLG,OPS
59,Aaron Judge,NYY,148,550,633,158,95,24,0,39,...,3,5,0,16,6,1,0.287,0.373,0.544,0.916


Compare two players' stats

In [13]:
def compareTwo (player1, player2):
  searchArray = [list(bi).index(player1), list(bi).index(player2)]
  return btc.loc[searchArray]

search1 = input("Enter a player's full name to search for: ")
search2 = input("Enter another player's full name to search for: ")
compareTwo(search1, search2)

Enter a player's full name to search for: Cody Bellinger
Enter another player's full name to search for: Juan Soto


,Name,Team,G,AB,PA,H,1B,2B,3B,HR,...,HBP,SF,SH,GDP,SB,CS,AVG,OBP,SLG,OPS
601,Cody Bellinger,LAD,95,315,350,52,31,9,2,10,...,1,3,0,2,3,1,0.165,0.240,0.302,0.542
32,Juan Soto,WSN,151,502,654,157,106,20,2,29,...,2,5,0,23,9,7,0.313,0.465,0.534,0.999


Search for player by name and a specific stat by abbreviation

In [14]:
#exception handling for input of nonexistent player/stat
try:
  searchName = input("Enter a player's full name to search for: ")
  searchStat = input("Enter a stat to view (use abbreviation): ")
  print(round(btc.loc[list(bi).index(searchName)].at[searchStat], 3)) #rounds stats to 3 decimal places
except ValueError:
  print('This player does not exist.')
except KeyError:
  print('This stat does not exist.')

Enter a player's full name to search for: Francisco Lindor
Enter a stat to view (use abbreviation): BB
58


Use stored stats to calculate other stats (HR/PA = home run percentage)

In [15]:
aaron_judge_hr = round(btc.loc[list(bi).index('Aaron Judge')].at['HR'], 3)
aaron_judge_pa = round(btc.loc[list(bi).index('Aaron Judge')].at['PA'], 3)
aaron_judge_hrpercent = round((aaron_judge_hr / aaron_judge_pa)*100, 3)
aaron_judge_hrpercent

6.161

Method for finding all players' HR/PA

In [16]:
def find_hrPercent():
  hrPercent = []
  for player in bi:
    hr = btc.loc[list(bi).index(player)].at['HR']
    pa = btc.loc[list(bi).index(player)].at['PA']
    hrpa = round((hr / pa)*100, 3)
    hrPercent.append(hrpa)
  return hrPercent

Add HR% to each player's profile in the DataFrame

In [17]:
#if statement helps prevent duplicate columns being created
if("HR%" not in btc.columns):
  btc.insert(btc.columns.size, 'HR%', find_hrPercent(), True)
btc

,Name,Team,G,AB,PA,H,1B,2B,3B,HR,...,SF,SH,GDP,SB,CS,AVG,OBP,SLG,OPS,HR%
0,Tommy Hunter,NYM,4,1,1,1,1,0,0,0,...,0,0,0,0,0,1.0,1.0,1.0,2.0,0.0
1,Daniel Bard,COL,67,1,1,1,1,0,0,0,...,0,0,0,0,0,1.0,1.0,1.0,2.0,0.0
2,Connor Overton,- - -,9,1,1,1,1,0,0,0,...,0,0,0,0,0,1.0,1.0,1.0,2.0,0.0
3,Jose Quintana,- - -,29,1,2,1,1,0,0,0,...,0,0,0,0,0,1.0,1.0,1.0,2.0,0.0
4,Josh Lowe,TBR,2,1,2,1,1,0,0,0,...,0,0,0,1,0,1.0,1.0,1.0,2.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1503,Garrett Crochet,CHW,54,0,0,0,0,0,0,0,...,0,0,0,0,0,0.0,0.0,0.0,0.0,NaN
1504,Spencer Strider,ATL,2,1,2,0,0,0,0,0,...,0,1,0,0,0,0.0,0.0,0.0,0.0,0.0
1505,Kohei Arihara,TEX,10,2,2,0,0,0,0,0,...,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0
1506,Hyeon-jong Yang,TEX,12,0,0,0,0,0,0,0,...,0,0,0,0,0,0.0,0.0,0.0,0.0,NaN


Display bar chart for top players for a calculated stat

In [18]:
interact(bar_plotB, stat=list(btc.columns)[4:], direction = ['Ascending', 'Descending'])

interactive(children=(Dropdown(description='stat', options=('PA', 'H', '1B', '2B', '3B', 'HR', 'R', 'RBI', 'BB…

<function __main__.bar_plotB(stat, direction)>

Calculate overall stats of all players combined

In [19]:
btc.describe()

,G,AB,PA,H,1B,2B,3B,HR,R,RBI,...,SF,SH,GDP,SB,CS,AVG,OBP,SLG,OPS,HR%
count,1508.000000,1508.000000,1508.000000,1508.000000,1508.000000,1508.000000,1508.000000,1508.000000,1508.000000,1508.000000,...,1508.000000,1508.000000,1508.000000,1508.000000,1508.000000,1508.000000,1508.00000,1508.000000,1508.000000,1052.000000
mean,47.494032,107.387931,120.568302,26.181034,16.580902,5.214191,0.444297,3.941645,14.595491,13.921088,...,0.757958,0.507958,2.208223,1.467507,0.471485,0.118119,0.16092,0.185952,0.346890,1.765038
std,44.458050,170.042510,191.144057,44.690097,28.425634,9.155365,1.082625,8.021325,25.606450,24.979535,...,1.658793,1.374453,4.078088,4.094391,1.276029,0.133601,0.17142,0.209838,0.366694,2.375005
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
25%,12.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
50%,31.000000,6.000000,7.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.071000,0.14300,0.089500,0.250000,0.834000
75%,69.000000,164.000000,184.000000,35.000000,22.250000,7.000000,0.000000,4.000000,19.000000,17.000000,...,1.000000,0.000000,3.000000,1.000000,0.000000,0.236000,0.30725,0.373000,0.679250,3.067000
max,162.000000,664.000000,724.000000,195.000000,136.000000,42.000000,8.000000,48.000000,123.000000,121.000000,...,12.000000,14.000000,28.000000,47.000000,10.000000,1.000000,1.00000,1.333000,2.000000,33.333000


# Pitcher Functions

Display pitchers

In [20]:
display(ptc)

,Name,Team,W,L,ERA,G,GS,CG,ShO,SV,...,H,R,ER,HR,BB,IBB,HBP,WP,BK,SO
0,Tommy Hunter,NYM,0,0,0.0,4,1,0,0,0,...,4,0,0,0,3,0,1,1,0,6
1,Oliver Perez,CLE,0,1,0.0,5,0,0,0,0,...,5,1,0,0,1,1,0,0,0,4
2,Mitch Moreland,OAK,0,0,0.0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
3,Charlie Culberson,TEX,0,0,0.0,2,0,0,0,0,...,2,0,0,0,0,0,0,0,0,0
4,Anthony Rizzo,CHC,0,0,0.0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
904,Robel Garcia,HOU,0,0,36.0,1,0,0,0,0,...,5,4,4,2,0,0,0,0,0,0
905,Wilmer Difo,PIT,0,0,36.0,2,0,0,0,0,...,9,8,8,1,3,0,0,0,0,1
906,Felix Pena,LAA,0,0,37.8,2,0,0,0,0,...,7,7,7,0,4,0,0,1,0,2
907,John Axford,MIL,0,0,54.0,1,0,0,0,0,...,2,2,2,0,1,0,1,0,0,0


Bar chart for top STARTERS for a chosen stat

In [21]:
def bar_plotStarters(stat, direction):
  pt_sorted = starters
  pt_qual = starters.loc[ptc['IP'] > 30]
  if direction == 'Ascending':
    directionB = False
  else:
    directionB = True
  pt_sorted = pt_qual.sort_values(by= stat, ascending= directionB)
  fig = px.bar(pt_sorted.head(100), x='Name', y=stat, width=1300, height=600)
  fig.update_layout(title_text="Top Starters in Pitching Stats in MLB (2021)")
  fig.update_xaxes(rangeslider_visible=True)
  fig.show()

interact(bar_plotStarters, stat=list(ptc.columns)[2:], direction = ['Ascending', 'Descending'])

interactive(children=(Dropdown(description='stat', options=('W', 'L', 'ERA', 'G', 'GS', 'CG', 'ShO', 'SV', 'HL…

<function __main__.bar_plotStarters(stat, direction)>

Bar chart for top RELIEVERS for a chosen stat

In [22]:
def bar_plotRelievers(stat, direction):
  pt_sorted = relievers
  pt_qual = relievers.loc[ptc['IP'] > 30]
  if direction == 'Ascending':
    directionB = False
  else:
    directionB = True
  pt_sorted = pt_qual.sort_values(by= stat, ascending= directionB)
  fig = px.bar(pt_sorted.head(100), x='Name', y=stat, width=1300, height=600)
  fig.update_layout(title_text="Top Starters in Pitching Stats in MLB (2021)")
  fig.update_xaxes(rangeslider_visible=True)
  fig.show()

interact(bar_plotRelievers, stat=list(ptc.columns)[2:], direction = ['Ascending', 'Descending'])

interactive(children=(Dropdown(description='stat', options=('W', 'L', 'ERA', 'G', 'GS', 'CG', 'ShO', 'SV', 'HL…

<function __main__.bar_plotRelievers(stat, direction)>